In [2]:
import sqlite3
import pandas as  pd
import numpy as np
import datetime
import os

### <span style="color:#9932CC">**1. Diretórios Mapeados**</span>

In [3]:
result_file ='../../Data/Modeling/score_sin.parquet'

path_database = '../../Code/Operationalization/db.sqlite3'

### <span style="color:#9932CC">**2. Leitura dos dados tratados**</span>

In [4]:
dataframe = pd.read_parquet(result_file)
dataframe.head(2)

,id_sin,dsc_texto,tip_supervisao,din_execucao,tip_evento,tip_predicao,pct_predicao
0,801877,substituicao do mangote da linha de fechament...,oper,2020-09-07 01:21:08.006362,True,True,100.0
1,801878,correcao de ponto quente lado fonte barra ...,oper,2020-09-07 01:21:08.006362,True,True,27.0


### <span style="color:#9932CC">**3. Parametros Query: Data**</span>

In [5]:
df_data = pd.DataFrame(data=dataframe.din_execucao.astype(str))
df_data = df_data.min()

array_df_data = df_data.to_numpy()
array_df_data

array(['2020-09-07 01:21:08.006362'], dtype=object)

### <span style="color:#9932CC">**3. Conversao para Array de Dataframe Completo**</span>

In [6]:
array_dataframe = dataframe.to_numpy()

### <span style="color:#9932CC">**4. Salva Dados em Database**</span>

In [7]:
sqliteConnection = sqlite3.connect(path_database)

cursor = sqliteConnection.cursor()
print("Conectado com sucesso ao SQLite")


#Insere informações no banco de dados
try:
    sqlite_insert_query = """INSERT INTO applications_dadosretreino(id_sin,dsc_texto,tip_supervisao,din_execucao,tip_evento,tip_predicao,pct_predicao) 
                             VALUES  (?, ?, ?, ?, ?, ? , ?) """

    cursor.executemany(sqlite_insert_query, array_dataframe)
    sqliteConnection.commit()
    print("Total(Teste-Treino) de registros Inseridos: ", cursor.rowcount)
    
    sqlite_update_query = """UPDATE applications_dadosretreino set pct_predicao = CAST(CAST(pct_predicao AS DECIMAL(4,2)) AS VARCHAR(10))"""
    cursor.execute(sqlite_update_query)
    sqliteConnection.commit()   
    
    cursor.close()
               
except sqlite3.Error as error:
    print("Falha ao inserir registros no database", error)
finally:
   if (sqliteConnection):
       sqliteConnection.close()
   print("Conexão com SQLite fechada")


Conectado com sucesso ao SQLite
Total(Teste-Treino) de registros Inseridos:  4998
Conexão com SQLite fechada
